# 이론 정리

# 라이브러리 임포트

In [13]:
# google drive mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
## for from kobert_transformers.utils import get_tokenizer
!pip install kobert-transformers

In [5]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
!pip install mxnet # 파이썬에서 실행 안 됨
!pip install transformers
!pip install torch

In [6]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
#!pip install onnxruntime==1.8.0
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install boto3

fatal: destination path 'KoBERT' already exists and is not an empty directory.
/content/KoBERT


In [7]:
## for import gluonnlp as nlp
!pip install gluonnlp

In [7]:
!pip install --upgrade numpy # upgrade numpy to the latest version if needed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.2.1 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.1 which is incompatible.
langchain 0.3.14 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.1 which is incompatible.
mxnet 1.9.1 requires numpy<2.0.0,>1.16.0, but you have numpy 2.2.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.1 which is incompatible.
pytensor 2.26.4 requires numpy<2,>

In [10]:
!pip install numpy==1.23.5 # Install a compatible version of numpy
!pip install "mxnet<2.0.0" # Install a compatible version of mxnet, or downgrade to below 2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.31.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.

In [8]:
# import modules

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

import numpy as np

from tqdm import tqdm, tqdm_notebook
from kobert_transformers.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


In [11]:
# for 데이터프레임 내 non-string을 string으로 변환
import pandas as pd
from transformers import BertModel, BertTokenizer
from kobert_tokenizer import KoBERTTokenizer  # 1-3-2. BERTDataset 형식 변환 바로 밑 설치 코드 실행 후 실행하면 잘 돌아감

In [10]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-og44lr3n/kobert-tokenizer_722d9a89ae61439f90e58fcc1ac0554a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-og44lr3n/kobert-tokenizer_722d9a89ae61439f90e58fcc1ac0554a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=dddaf2b22af3724c2291d54e55d11b10c89ed8587931ed47959174d0e0479d09
  Stored in directory: /tmp/pip-ephem-wheel-cache-x97e55ux/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_tdiil46
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_tdii

# 로직 구현

In [12]:
# BERT 모델과 Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()
MAX_LEN = 64

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [34]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        # TorchScript 호환 방식으로 변경
        batch_size, seq_len = token_ids.size()
        attention_mask = torch.arange(seq_len, device=token_ids.device).expand(batch_size, seq_len)
        attention_mask = (attention_mask < valid_length.unsqueeze(1)).float()
        return attention_mask

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(
            input_ids=token_ids,
            token_type_ids=segment_ids.long(),
            attention_mask=attention_mask.float().to(token_ids.device),
            return_dict=False,
        )
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
# set device: PyTorch에서 GPU를 사용하기 위한 설정을 지정
# device = torch.device("cuda:0")
# print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [39]:
model_path = "/content/gdrive/MyDrive/crimefin/model/model.pt"
state_dict = torch.load(model_path, map_location="cpu")
print("state_dict keys:", state_dict.keys())


<ipython-input-39-017194d53447>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location="cpu")


state_dict keys: dict_keys(['config', 'bert', 'classes'])


In [40]:
# 학습된 모델 로드
model_path = "/content/gdrive/MyDrive/crimefin/model/model.pt"
state_dict = torch.load(model_path, map_location=device)

# 'bert' 키에서 실제 state_dict 추출
bert_state_dict = state_dict['bert']

<ipython-input-40-bfc5488198cb>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)


In [41]:
# 저장된 모델의 클래스 수를 기반으로 BERTClassifier 생성
trained_model = BERTClassifier(bertmodel, num_classes=2, dr_rate=0.5).to(device)
trained_model.load_state_dict(bert_state_dict, strict=False)
trained_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [45]:
# TorchScript 변환을 위한 입력 데이터 생성
example_token_ids = torch.randint(0, 30522, (1, MAX_LEN), device=device)
# Clamp token IDs to be within the valid range for KoBERT vocabulary
example_token_ids = torch.clamp(example_token_ids, min=0, max=vocab.token_to_idx['[MASK]']) # 'MASK' token is usually the last token
valid_length = (example_token_ids != 0).sum(dim=1)
example_valid_length = valid_length.to(device)
example_segment_ids = torch.zeros((1, MAX_LEN), dtype=torch.long).to(device)

In [43]:
print("Token IDs:", example_token_ids)
print("Valid Length:", example_valid_length)
print("Segment IDs:", example_segment_ids)


Token IDs: tensor([[15597, 23956, 11284, 24581, 29294,  7279, 22066, 29227,   977,  8530,
         18879,  1347, 25300, 26154, 19773, 24266, 25569, 21740, 29171, 28196,
         14934, 17748, 26940, 25331,  8065,  8538, 22359, 12633, 21260, 13430,
         11662,  3670, 26144, 13161,  5130,  5825,  7307, 26640, 15487,  3648,
          3723,  4456, 21658, 22410, 14337, 14025,  7640, 25559, 23671,   411,
          6659, 25941, 23285, 16908, 22909, 19482, 18022, 22304,  1872, 16029,
         10795,  1363, 15740, 21891]])
Valid Length: tensor([64])
Segment IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [37]:
# 데이터 검증
if not torch.all((example_token_ids >= 0) & (example_token_ids < 30522)):
    raise ValueError(f"Token IDs contain values outside the valid range [0, 30521]: {example_token_ids}")

In [51]:
# TorchScript 변환
scripted_model = torch.jit.trace(
    trained_model, (example_token_ids, example_valid_length, example_segment_ids)
)
scripted_model.save("../gdrive/MyDrive/crimefin/model/torchscript_kobert.pt")
print("TorchScript 모델 저장 완료")

TorchScript 모델 저장 완료


In [47]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: /content/KoBERT
